# Global Capital Markets Project
Project by Alexander Jermann, Columbia University under supervision of Prof. Dr. Siddhartha Dastidar, Columbia University

### Table of Contents
This notebook was used to pre-process all the 10-K and 10-Q reports in order to compute the similarity measures between subsequent 10-K and 10-Q reports. Running this notebook took a lot longer than I had anticipated for various reasons. First, the 10-K and 10-Q reports for over 2000 companies over 10 years took around 4 hours to download and another 6 hours to unzip. Second, the dataset requires around 60-100 gigabytes of storage. Third, computing similarity measures between so many documents also requires around 4 hours. For convenience, I saved all the intermediary steps as a CSV file and will save them together with all the datasets to a USB drive that I will also hand-in. 

This notebook contains all the code that was used in this project to:
1. List of all U.S. equities.
2. Map the U.S. Stocks to the Central Index Key (CIK) that the S.E.C. uses internally. 
3. Compilation of a CSV containing all the 10-K and 10-Q file summaries and clean-up statistics.
4. Defining Similarity measures
5. Computing similarities between 10-K reports
6. Computing similarities between 10-Q reports (to corresponding quarter of the following year)
7. Transposing statistics in a appropriate format for Alphawise.

Note 1: The second part of the project is in a second notebook which is run on a platform Quantopian, that allows to run the factor models online. Please see Notebook 2.

Note 2: The locations of the files in the notebook, might have to be adjusted to fit the own working environment. 

### Data
The following files are available on the github repository:
1. 
The data can be downloaded from the following website [https://sraf.nd.edu/data/stage-one-10-x-parse-data/](https://sraf.nd.edu/data/stage-one-10-x-parse-data/)

In [1]:
# Importing built-in libraries (no need to install these)
import re
import os
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata

# Importing libraries you need to install
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from iexfinance.stocks import get_historical_data
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm

## Get all Stock tickers of US Equities

In this section we get a list of the stock tickers of all US Equities. More specifially, a list of all stocks (historical and present) traded on the New York Stock Exchange ("NYSE") on the National Association of Securities Dealers ("NASDAQ"), and the American Stock Exchange ("AMEX"). We do that by downloading the CSV's from the respective sites and then loading them using Pandas dataframes.

CSV downloaded from respective websites (Please note: website address might change in the future):
- NASDQ: https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nasdaq&render=download
- AMEX: https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=amex&render=download
- NYSE:https://old.nasdaq.com/screening/companies-by-name.aspx?letter=0&exchange=nyse&render=download

In [74]:
# enter the location of the saved file in the comamand below
nasdaq = pd.read_csv('data/symbols_nasdaq.csv')
nyse = pd.read_csv('data/symbols_nyse.csv')
amex = pd.read_csv('data/symbols_amex.csv')

In [75]:
nasdaq.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,TXG,"10x Genomics, Inc.",64.8900,$6.24B,2019.0,Capital Goods,Biotechnology: Laboratory Analytical Instruments,https://old.nasdaq.com/symbol/txg,NaN
1,YI,"111, Inc.",5.1100,$417.28M,2018.0,Health Care,Medical/Nursing Services,https://old.nasdaq.com/symbol/yi,NaN
2,PIH,"1347 Property Insurance Holdings, Inc.",4.6400,$27.93M,2014.0,Finance,Property-Casualty Insurers,https://old.nasdaq.com/symbol/pih,NaN
3,PIHPP,"1347 Property Insurance Holdings, Inc.",25.7615,$18.03M,NaN,Finance,Property-Casualty Insurers,https://old.nasdaq.com/symbol/pihpp,NaN
4,TURN,180 Degree Capital Corp.,2.1750,$67.69M,NaN,Finance,Finance/Investors Services,https://old.nasdaq.com/symbol/turn,NaN


In [76]:
nyse.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,DDD,3D Systems Corporation,8.49,$1.01B,NaN,Technology,Computer Software: Prepackaged Software,https://old.nasdaq.com/symbol/ddd,NaN
1,MMM,3M Company,169.77,$97.63B,NaN,Health Care,Medical/Dental Instruments,https://old.nasdaq.com/symbol/mmm,NaN
2,WBAI,500.com Limited,9.55,$410.41M,2013.0,Consumer Services,Services-Misc. Amusement & Recreation,https://old.nasdaq.com/symbol/wbai,NaN
3,WUBA,58.com Inc.,61.52,$9.15B,2013.0,Technology,"Computer Software: Programming, Data Processing",https://old.nasdaq.com/symbol/wuba,NaN
4,EGHT,8x8 Inc,20.52,$2.06B,NaN,Technology,EDP Services,https://old.nasdaq.com/symbol/eght,NaN


In [77]:
amex.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote,Unnamed: 8
0,XXII,"22nd Century Group, Inc",1.030,$137.12M,NaN,Consumer Non-Durables,Farming/Seeds/Milling,https://old.nasdaq.com/symbol/xxii,NaN
1,FAX,Aberdeen Asia-Pacific Income Fund Inc,4.280,$1.06B,1986.0,NaN,NaN,https://old.nasdaq.com/symbol/fax,NaN
2,IAF,Aberdeen Australia Equity Fund Inc,5.430,$123.49M,NaN,NaN,NaN,https://old.nasdaq.com/symbol/iaf,NaN
3,AEF,"Aberdeen Emerging Markets Equity Income Fund, ...",7.091,$359.88M,NaN,NaN,NaN,https://old.nasdaq.com/symbol/aef,NaN
4,FCO,"Aberdeen Global Income Fund, Inc.",7.850,$68.49M,1992.0,NaN,NaN,https://old.nasdaq.com/symbol/fco,NaN


In [78]:
# Delete uneeded columns
nasdaq.drop(labels='Unnamed: 8', axis='columns', inplace=True)
nyse.drop(labels='Unnamed: 8', axis='columns', inplace=True)
amex.drop(labels='Unnamed: 8', axis='columns', inplace=True)

# Merge all into a single list 
tickers = list(set(list(nasdaq['Symbol']) + list(nyse['Symbol']) + list(amex['Symbol'])))

In [80]:
# Example of 10 items from list
tickers[:10]

['ELOX', 'VCLT', 'DS^B', 'UNTY', 'FR', 'KERN', 'MIC', 'MET', 'RFIL', 'BA']

## Map Stock Symbols to SEC CIK

This next section downloads the list of stock symbols to CIK mapping.

In [81]:
def MapStockToCik(tickers):
    url = 'http://www.sec.gov/cgi-bin/browse-edgar?CIK={}&Find=Search&owner=exclude&action=getcompany'
    cik_re = re.compile(r'.*CIK=(\d{10}).*')

    cik_dict = {}
    for ticker in tqdm(tickers):
        results = cik_re.findall(requests.get(url.format(ticker)).text)
        if len(results):
            cik_dict[str(ticker).lower()] = str(results[0])
    
    return cik_dict

Note: the below code cell, took over an hour to run. For convenience saved results as a CSV file titled: `'ticker_cik.csv'`

In [84]:
cik_dict = MapStockToCik(tickers)

100%|██████████| 6989/6989 [1:08:21<00:00,  1.70it/s]


In [86]:
# Clean-up the mapping as a DataFrame
ticker_cik_df = pd.DataFrame.from_dict(data=cik_dict, orient='index')
ticker_cik_df.reset_index(inplace=True)
ticker_cik_df.columns = ['ticker', 'cik']
ticker_cik_df['cik'] = [str(cik) for cik in ticker_cik_df['cik']]

In [314]:
ticker_cik_df

,ticker,cik
2025,a,0001090872
462,aa,0001675149
2159,aacg,0001420529
3011,aal,0000006201
3896,aamc,0001555074
...,...,...
4179,zumz,0001318008
5011,zuo,0001423774
1415,zvo,0001305323
943,zyme,0001403752


In [88]:
ticker_cik_df.to_csv('data/ticker_cik.csv')

In [89]:
print("Number of ticker-cik pairings:", len(ticker_cik_df))
print("Number of unique tickers:", len(set(ticker_cik_df['ticker'])))
print("Number of unique CIKs:", len(set(ticker_cik_df['cik'])))

Number of ticker-cik pairings: 5040
Number of unique tickers: 5040
Number of unique CIKs: 4851


In [92]:
ticker_cik_df = ticker_cik_df.sort_values(by='ticker')
ticker_cik_df.drop_duplicates(subset='cik', keep='first', inplace=True)

In [93]:
print("Number of ticker-cik pairings:", len(ticker_cik_df))
print("Number of unique tickers:", len(set(ticker_cik_df['ticker'])))
print("Number of unique CIKs:", len(set(ticker_cik_df['cik'])))

Number of ticker-cik pairings: 4851
Number of unique tickers: 4851
Number of unique CIKs: 4851


In [385]:
ticker_cik_df['cik_short'] = ticker_cik_df['cik'].str.lstrip("0")

In [387]:
ticker_cik_df

,ticker,cik,cik_short
2025,a,0001090872,1090872
462,aa,0001675149,1675149
2159,aacg,0001420529,1420529
3011,aal,0000006201,6201
3896,aamc,0001555074,1555074
...,...,...,...
4179,zumz,0001318008,1318008
5011,zuo,0001423774,1423774
1415,zvo,0001305323,1305323
943,zyme,0001403752,1403752


Save the cleaned-up mapping as a CSV file.

In [386]:
ticker_cik_df.to_csv('data/ticker_cik_clean.csv')

## Compile File Summary
In this section we compile a file summary that maps the stock ticker to the CIK and to the location of the file on the disk. This file also contains information on the filing date, the form type (i.e. 10-K or 10-Q), information on the sentiment of the documents (more detail to this in later section), and statistics on the clean-up (i.e. how many tables and exhibits, how many HTML tags were removed etc.). For a full list see the command `df1.columns` below.

In [ ]:
# replace string below the location of the file
df1 = pd.read_csv('data/LM_10X_Summaries_2018.csv') 

In [ ]:
df1.columns

In [ ]:
# replace string with the location of the 10-K and 10-Q file locations
filename_column = df1['FILE_NAME'].str.slice_replace(start=0, stop=16, repl='/Volumes/Alexander/Columbia/')
filename_column = filename_column.str.replace("\\", "/")

In [ ]:
df1['FILE_LOCATION'] = filename_column

In [ ]:
df1.drop('FILE_NAME', axis=1, inplace=True)

In [ ]:
# replace below command with the location that you wish to save the command
df1.to_csv('data/10X_Summaries_AJ.csv')

To read file we have ro specify `index_col` as follows:

In [108]:
# enter location where saved csv in previous step
fs = pd.read_csv('data/10X_Summaries_AJ.csv', index_col=0)

In [109]:
fs.head()

,CIK,FILING_DATE,FYE,FORM_TYPE,SIC,FFInd,N_Words,N_Unique_Words,N_Negative,N_Positive,...,N_Negation,GrossFileSize,NetFileSize,NonTextDocTypeChars,HTMLChars,XBRLChars,XMLChars,N_Tables,N_Exhibits,FILE_LOCATION
0,60512,19930813,19930630,10-Q,1311,30,3037,700,68,6,...,1,63255,25233,0,186,0,0,7,0,/Volumes/Alexander/Columbia/1993/QTR3/19930813...
1,66740,19930813,19930630,10-Q,2670,38,4102,823,53,24,...,3,57283,47739,0,317,0,0,2,3,/Volumes/Alexander/Columbia/1993/QTR3/19930813...
2,60512,19931007,19921231,10-K-A,1311,30,7278,1184,157,41,...,5,114136,56958,0,276,0,0,22,0,/Volumes/Alexander/Columbia/1993/QTR4/19931007...
3,60512,19931110,19930930,10-Q,1311,30,3768,739,69,13,...,1,72062,31256,0,201,0,0,8,0,/Volumes/Alexander/Columbia/1993/QTR4/19931110...
4,11860,19931112,19930930,10-Q,3312,19,3822,897,77,39,...,2,40392,39056,0,201,0,0,0,0,/Volumes/Alexander/Columbia/1993/QTR4/19931112...


In [113]:
fs[fs['FILING_DATE'] > 20110000]

,CIK,FILING_DATE,FYE,FORM_TYPE,SIC,FFInd,N_Words,N_Unique_Words,N_Negative,N_Positive,...,N_Negation,GrossFileSize,NetFileSize,NonTextDocTypeChars,HTMLChars,XBRLChars,XMLChars,N_Tables,N_Exhibits,FILE_LOCATION
768249,1006820,20110103,20091231,10-K-A,6022,44,5314,891,25,24,...,3,326459,46894,0,227014,0,0,5,4,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
768250,1084448,20110103,20091231,10-K-A,4813,32,13797,1532,135,42,...,11,1682775,109692,0,1433786,0,0,51,4,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
768251,1259550,20110103,20091231,10-K-A,7389,34,50487,2936,1029,387,...,45,937469,358399,0,359374,0,0,53,8,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
768252,1352482,20110103,20091130,10-K-A,2834,13,15055,1669,162,69,...,13,2139648,116317,1741462,189583,0,0,12,4,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
768253,1443157,20110103,20100531,10-K-A,3824,37,7712,1304,134,40,...,2,518426,57194,0,421239,0,0,17,2,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028669,1363343,20181231,20180930,10-Q,6770,47,4072,806,51,14,...,3,1106870,33975,80161,318492,378675,268010,11,8,/Volumes/Alexander/Columbia/2018/QTR4/20181231...
1028670,1497251,20181231,20180930,10-Q,7389,34,6403,1039,119,32,...,9,1092655,50263,79621,254605,330332,311590,10,10,/Volumes/Alexander/Columbia/2018/QTR4/20181231...
1028671,1510247,20181231,20180930,10-Q,7374,34,15359,1908,236,97,...,10,2175983,114219,171325,486263,815462,547824,15,11,/Volumes/Alexander/Columbia/2018/QTR4/20181231...
1028672,1602813,20181231,20180930,10-Q,8742,34,5021,868,78,25,...,4,814265,42154,59433,203907,225746,219529,10,10,/Volumes/Alexander/Columbia/2018/QTR4/20181231...


## Similarity Measures

This section defines the Jaccard and Cosine similarity measures between two words. We define them as  follows:

**Jaccard Similarity Index:**
Let A and B be words, then the Jaccard Similarity score is
$$
J(A, B) = \frac{\mid A \cap B \mid}{\mid A \cup B \mid}
$$

where $0 \geq J(A, B) \geq 1$. If both $A,B = \emptyset$ we define $J(A, B) = 1$.

**Cosine Similarity Measure:**
Let $A$ and $B$ be words. We map $A$ and $B$ into the vector space $S$, where $S$ has dimension of the union between the word sets $A$ and $B$ 


The dimensions of the vectorspace are the set of the union between A and B. The vector $A \in $
$$
C(A, B) = \frac{\mathbf{A} \cdot \mathbf{B}}{\mid \mid \mathbf{A} \mid \mid \cdot \mid \mid \mathbf{B} \mid \mid}
$$

where the nominator is the dot product between the vectors $\mathbf{A}$ and $\mathbf{B}$ and the denominator is the Euclidean norm. 

Inputs
- whole documents
- compare 10ks
- compare 10qs

### Using word sets

In [307]:
def jaccard_similarity(words_A, words_B):
    
    # Count number of words in both A and B
    words_intersect = len(words_A.intersection(words_B))
    
    # Count number of words in A or B
    words_union = len(words_A.union(words_B))
    
    # Compute Jaccard similarity score
    jaccard_score = words_intersect / words_union
    
    return jaccard_score

In [220]:
def c_cosine_similarity(words_A, words_B):
    
    # Get the union of words between A and B
    words = list(words_A.union(words_B))
    
    # Figure out which words are in A
    vector_A = [1 if x in words_A else 0 for x in words]
    
    # Figure out which words are in B
    vector_B = [1 if x in words_B else 0 for x in words]
    
    # Calculate cosine score using scikit-learn package
    array_A = np.array(vector_A).reshape(1, -1)
    array_B = np.array(vector_B).reshape(1, -1)
    cosine_score = cosine_similarity(array_A, array_B)[0,0]
    
    return cosine_score

As an exanple of how the two similarity measures work, let's consider the following three sentences as sets of words:

In [221]:
d_a = set(['we', 'expect', 'demand', 'to', 'increase','increase'])
d_b = set(['we', 'expect', 'worldwide', 'demand', 'to', 'increase'])
d_c = set(['we', 'expect', 'weakness', 'in', 'sales'])

In [308]:
print("Cosine similarity between A and B:", c_cosine_similarity(d_a, d_b))
print("Cosine similarity between A and C:", c_cosine_similarity(d_a, d_c))
print("Jaccard similarity between A and B:", ComputeJaccardSimilarity(d_a, d_b))
print("Jaccard similarity between A and C:", ComputeJaccardSimilarity(d_a, d_c))

Cosine similarity between A and B: 0.912870929175277
Cosine similarity between A and C: 0.39999999999999997
Jaccard similarity between A and B: 0.8333333333333334
Jaccard similarity between A and C: 0.25


### Using TF-IDF
TF-IDF: stands for term frequency - inverse document frequency

In [217]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [219]:
vectorizer = TfidfVectorizer()
vectorizer.fit_transform(d_a)

vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [ ]:
# Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

## Computing Similarities for 10Ks

Here we get a CSV file that contains information and the location of all the necessary files. 

In [72]:
# replace with location of the summary file. 
fs = pd.read_csv('data/10X_Summaries_AJ.csv', index_col=0)

/Users/alexander/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [74]:
fs[(fs['FORM_TYPE'].str.startswith('10-K')) & (fs['FILING_DATE'] > 20110000)]

,CIK,FILING_DATE,FYE,FORM_TYPE,SIC,FFInd,N_Words,N_Unique_Words,N_Negative,N_Positive,...,N_Negation,GrossFileSize,NetFileSize,NonTextDocTypeChars,HTMLChars,XBRLChars,XMLChars,N_Tables,N_Exhibits,FILE_LOCATION
768249,1006820,20110103,20091231,10-K-A,6022,44,5314,891,25,24,...,3,326459,46894,0,227014,0,0,5,4,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
768250,1084448,20110103,20091231,10-K-A,4813,32,13797,1532,135,42,...,11,1682775,109692,0,1433786,0,0,51,4,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
768251,1259550,20110103,20091231,10-K-A,7389,34,50487,2936,1029,387,...,45,937469,358399,0,359374,0,0,53,8,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
768252,1352482,20110103,20091130,10-K-A,2834,13,15055,1669,162,69,...,13,2139648,116317,1741462,189583,0,0,12,4,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
768253,1443157,20110103,20100531,10-K-A,3824,37,7712,1304,134,40,...,2,518426,57194,0,421239,0,0,17,2,/Volumes/Alexander/Columbia/2011/QTR1/20110103...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028662,1636051,20181231,20180930,10-K,1000,28,20448,2083,330,140,...,15,2536971,152534,205897,626233,817250,596068,16,12,/Volumes/Alexander/Columbia/2018/QTR4/20181231...
1028663,1663641,20181231,20180930,10-K,7310,34,10679,1504,126,59,...,11,1325731,83282,161072,266479,300795,330016,14,8,/Volumes/Alexander/Columbia/2018/QTR4/20181231...
1028664,1704795,20181231,20180930,10-K,3721,24,61971,3473,1521,352,...,40,6940663,448617,457361,1534367,2339415,2049698,27,9,/Volumes/Alexander/Columbia/2018/QTR4/20181231...
1028665,1730773,20181231,20181031,10-K,2092,2,31030,2726,683,205,...,21,2035116,227041,124000,715575,495121,386590,92,11,/Volumes/Alexander/Columbia/2018/QTR4/20181231...


In [366]:
ticker_cik_df

,ticker,cik
2025,a,0001090872
462,aa,0001675149
2159,aacg,0001420529
3011,aal,0000006201
3896,aamc,0001555074
...,...,...
4179,zumz,0001318008
5011,zuo,0001423774
1415,zvo,0001305323
943,zyme,0001403752


In [416]:
filenames = fs[(fs['CIK'] == 1675149) & (fs['FORM_TYPE'].str.startswith('10-K')) & (fs['FILING_DATE'] > 20110000)]['FILE_LOCATION']

In [409]:
filenames.sort()

In [417]:
filenames

977832     /Volumes/Alexander/Columbia/2017/QTR1/20170315...
1003695    /Volumes/Alexander/Columbia/2018/QTR1/20180226...
Name: FILE_LOCATION, dtype: object

In [436]:
fs[fs['CIK'].isin([cik])]

,CIK,FILING_DATE,FYE,FORM_TYPE,SIC,FFInd,N_Words,N_Unique_Words,N_Negative,N_Positive,...,N_Negation,GrossFileSize,NetFileSize,NonTextDocTypeChars,HTMLChars,XBRLChars,XMLChars,N_Tables,N_Exhibits,FILE_LOCATION
972285,1675149,20161201,20160930,10-Q,3350,19,27771,2123,620,193,...,16,7728707,215865,472918,1321347,3501582,1927234,35,10,/Volumes/Alexander/Columbia/2016/QTR4/20161201...
977832,1675149,20170315,20161231,10-K,3350,19,93479,4176,2126,640,...,54,19505575,727358,1165487,5083863,8367675,3785249,104,13,/Volumes/Alexander/Columbia/2017/QTR1/20170315...
984564,1675149,20170510,20170331,10-Q,3350,19,30910,2566,617,204,...,24,5982193,233165,394135,962604,2617223,1594743,27,11,/Volumes/Alexander/Columbia/2017/QTR2/20170510...
989535,1675149,20170803,20170630,10-Q,3350,19,48603,3154,988,364,...,53,7386320,357119,476399,1303226,3215561,1752559,35,14,/Volumes/Alexander/Columbia/2017/QTR3/20170803...
995228,1675149,20171027,20170930,10-Q,3334,19,26811,2190,553,205,...,16,7559202,208590,466410,1257802,3423219,1915109,35,10,/Volumes/Alexander/Columbia/2017/QTR4/20171027...
1003695,1675149,20180226,20171231,10-K,3334,19,133948,4639,2608,808,...,85,20826691,1005526,1208004,5151412,8859474,4226830,98,17,/Volumes/Alexander/Columbia/2018/QTR1/20180226...
1011899,1675149,20180509,20180331,10-Q,3334,19,25253,2344,515,177,...,17,5822790,194086,380144,908024,2658072,1497376,34,11,/Volumes/Alexander/Columbia/2018/QTR2/20180509...
1017113,1675149,20180802,20180630,10-Q,3334,19,25465,2172,500,175,...,16,7552958,197077,469918,1219258,3493893,1887964,36,10,/Volumes/Alexander/Columbia/2018/QTR3/20180802...
1023829,1675149,20181102,20180930,10-Q,3334,19,26166,2242,498,195,...,17,7994190,202380,502494,1306682,3686672,2011603,34,10,/Volumes/Alexander/Columbia/2018/QTR4/20181102...


In [443]:
# Replace the paths below with the location of the files
def computeSimilarity10K(cik, start_date=20110000):
    
    path = '/Users/alexander/git/gcm/'
    os.chdir(path)
    
    # Get filenames for given CIK
    filenames = fs[(fs['CIK'].isin([cik])) & (fs['FORM_TYPE'].str.startswith('10-K')) & (fs['FILING_DATE'] > 20110000)]['FILE_LOCATION']
    filenames = filenames.tolist()

    filenames.sort()
    
    # check if scores have already been calculated
    if os.path.exists('/Users/alexander/git/gcm/metrics/' + str(cik) + '_sim_scores.csv'):
        return
    
    # Check enough files
    if len(filenames) < 2:
        return 
    
    # Initialize dataframe to store sim scores
    dates = [x[38:46] for x in filenames]
    cosine_score = [0]*len(dates)
    jaccard_score = [0]*len(dates)
    data = pd.DataFrame(columns={'cosine_score': cosine_score, 
                                 'jaccard_score': jaccard_score},
                       index=dates)
    
    doc1_loc = filenames[0]
    
    with open(doc1_loc, 'r') as file:
        doc1_text = file.read()  
    
    for i in range(1, len(filenames)): # 

        doc2_loc = filenames[i]

        with open(doc2_loc, 'r') as file:
            doc2_text = file.read()

        # Get set of words in A, B
        words_A = set(re.findall(r"[\w']+", doc1_text))
        words_B = set(re.findall(r"[\w']+", doc2_text))

        # Calculate similarity scores
        cosine_score = c_cosine_similarity(words_A, words_B)
        jaccard_score = jaccard_similarity(words_A, words_B)

        # Store score values
        date_B = doc2_loc[38:46]
        data.at[date_B, 'cosine_score'] = cosine_score
        data.at[date_B, 'jaccard_score'] = jaccard_score
        
        doc1_text = doc2_text
        
    # Save to csv data
    os.chdir('/Users/alexander/git/gcm/metrics')
    data.to_csv(str(cik) + '_sim_scores.csv', index=False)
    os.chdir('/Users/alexander/git/gcm/')
    
    # Read in second 10K 
    # compute similarity
    # return similarity score

In [422]:
computeSimilarity10K(60512)

60512
Series([], Name: FILE_LOCATION, dtype: object)


In [444]:
tqdm._instances.clear()

In [445]:
#ticker_cik_df['cik']

cik_list = fs[(fs['FORM_TYPE'].str.startswith('10-K')) & (fs['FILING_DATE'] > 20110000)]['CIK']

for cik in tqdm(ticker_cik_df['cik_short']):
    computeSimilarity10K(cik)

100%|██████████| 4851/4851 [1:17:46<00:00,  1.18s/it]  


## Computing Similarities for 10-Qs

In [467]:
filenames = fs[(fs['CIK'].isin([1675149])) & (fs['FORM_TYPE'].str.startswith('10-Q')) & (fs['FILING_DATE'] > 20110000)]['FILE_LOCATION']

In [468]:
[datetime.strptime(x[38:46], '%Y%m%d').strftime('%Y-%m-%d') for x in filenames]

['2016-12-01',
 '2017-05-10',
 '2017-08-03',
 '2017-10-27',
 '2018-05-09',
 '2018-08-02',
 '2018-11-02']

In [475]:
# Replace the paths below with the location of the files
def computeSimilarity10Q(cik, start_date=20110000):

    # Define how stringent we want to be about
    # "previous year"
    year_short = timedelta(345)
    year_long = timedelta(385)

    # Set path
    path = '/Users/alexander/git/gcm/metrics'
    os.chdir(path)

    # Get filenames for given CIK
    filenames = fs[(fs['CIK'].isin([cik])) & (fs['FORM_TYPE'].str.startswith('10-Q')) & (fs['FILING_DATE'] > 20110000)]['FILE_LOCATION']
    filenames = filenames.tolist()
    filenames.sort()

    # check if scores have already been calculated
    if os.path.exists('/Users/alexander/git/gcm/metrics/' + str(cik) + '_sim_scores.csv'):
        return

    # Check if enough files exist to compare
    # ... if there aren't enough files, exit
    if len(filenames) < 4:
        #print("No files to compare for CIK", cik)
        os.chdir('../..')
        return

    # Initialize dataframe to hold similarity scores
    dates = [datetime.strptime(x[38:46], '%Y%m%d').strftime('%Y-%m-%d') for x in filenames]
    cosine_score = [0]*len(dates)
    jaccard_score = [0]*len(dates)
    data = pd.DataFrame(columns={'cosine_score': cosine_score,
                                 'jaccard_score': jaccard_score},
                       index=dates)

    # Iterate over each quarter...
    for j in range(3):

        # Get text and date of earliest filing from that quarter
        file_name_A = filenames[j]
        with open(file_name_A, 'r') as file:
            file_text_A = file.read()
        date_A = datetime.strptime(file_name_A[38:46], '%Y%m%d')

        # Iterate over the rest of the filings from that quarter...
        for i in range(j+3, len(filenames), 3):

            # Get name and date of the later file
            file_name_B = filenames[i]
            date_B = datetime.strptime(file_name_B[38:46], '%Y%m%d')

            # If B was not filed within ~1 year after A...
            if (date_B > (date_A + year_long)) or (date_B < (date_A + year_short)):

                #print(date_B.strftime('%Y-%m-%d'), "is not within a year of", date_A.strftime('%Y-%m-%d'))

                # Record values as NaN
                data.at[date_B.strftime('%Y-%m-%d'), 'cosine_score'] = 'NaN'
                data.at[date_B.strftime('%Y-%m-%d'), 'jaccard_score'] = 'NaN'

                # Pretend as if we found new date_A in the next year
                date_A = date_A.replace(year=date_B.year)

                # Move to next filing
                continue

            # If B was filed within ~1 year of A...

            # Get file text
            with open(file_name_B, 'r') as file:
                file_text_B = file.read()

            # Get sets of words in A, B
            words_A = set(re.findall(r"[\w']+", file_text_A))
            words_B = set(re.findall(r"[\w']+", file_text_B))

            # Calculate similarity score
            cosine_score = c_cosine_similarity(words_A, words_B)
            jaccard_score = jaccard_similarity(words_A, words_B)

            # Store value (indexing by the date of document B)
            data.at[date_B.strftime('%Y-%m-%d'), 'cosine_score'] = cosine_score
            data.at[date_B.strftime('%Y-%m-%d'), 'jaccard_score'] = jaccard_score

            # Reset value for next loop
            # Don't re-read files, for efficiency
            file_text_A = file_text_B
            date_A = date_B

    # Save scores
    data.to_csv(str(cik)+'_sim_scores.csv', index=True)


In [476]:
computeSimilarity10Q(1675149)

In [483]:
tqdm._instances.clear()

In [478]:
for cik in tqdm(ticker_cik_df['cik_short']):
    computeSimilarity10Q(cik)

100%|██████████| 4851/4851 [1:55:51<00:00,  1.26s/it]  


In [479]:
# Replace the paths below with the location of the files
def get_data(cik, pathname_data):
    
    data_10k = True
    data_10q = True
    
    path = '/Users/alexander/git/gcm/10K_metrics'
    os.chdir(path)
    
    try:
        df_10k_sim_score = pd.read_csv(str(cik) + '_sim_scores.csv')
    except FileNotFoundError:
        data_10k = False
        
        
    path = '/Users/alexander/git/gcm/10Q_metrics'
    os.chdir(path)
    
    try:
        df_10q_sim_score = pd.read_csv(str(cik) + '_sim_scores.csv')
    except FileNotFoundError:
        data_10q = False
        

    if not (data_10k and data_10q):
        return
    
    if not data_10q:
        sim_scores = df_10k_sim_score

    elif not data_10k:
        sim_scores = df_10q_sim_score

    elif (data_10q and data_10k):
        sim_scores = pd.concat([df_10k_sim_score, df_10q_sim_score], axis='index')
        

    sim_scores.rename(columns={'Unnamed: 0': 'date'}, inplace=True)


    sim_scores['cik'] = cik
    
    # Save file in the data dir
    os.chdir(pathname_data)
    sim_scores.to_csv('%s_sim_scores_full.csv' % cik, index=False)
    
    return

In [480]:
# Replace the paths below with the location of the files
pathname_data = '/Users/alexander/git/gcm/metrics_merged'

In [484]:
for cik in tqdm(ticker_cik_df['cik_short']):
    get_data(cik, pathname_data)

  0%|          | 0/4851 [00:00<?, ?it/s]/Users/alexander/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

100%|██████████| 4851/4851 [00:38<00:00, 124.68it/s]


In [485]:
def MakeDataset(file_list, pathname_full_data):
    
    '''
    Consolidates CIK datasets into a
    single dataset.
    
    Parameters
    ----------
    file_list : list
        List of .csv files to merge.
    pathname_full_data : str
        Path to directory to store
        full dataset.
        
    Returns
    -------
    None.
    
    '''
    
    # Initialize dataframe to store results
    data = pd.DataFrame(columns=['date', 'cosine_score', 'jaccard_score', 'cik'])
    
    # Iterate over files and merge all together
    for file_name in tqdm(file_list):
        new_data = pd.read_csv(file_name)
        data = data.append(new_data, sort=True)
    
    # Store result
    os.chdir(pathname_full_data)
    data.to_csv('all_sim_scores.csv', index=False)
    
    return

In [486]:
pathname_full_data = '/Users/alexander/git/gcm/final'

In [487]:
os.chdir(pathname_data)
file_list = [fname for fname in os.listdir() if not fname.startswith('.')]

MakeDataset(file_list, pathname_full_data)

100%|██████████| 3478/3478 [01:05<00:00, 53.26it/s]


In [494]:
sim_scores_full = pd.read_csv('all_sim_scores.csv')

# Cast CIKs as strings
sim_scores_full['cik'] = [str(x) for x in sim_scores_full['cik']]

# Merge to map tickers to CIKs
sim_scores_ticker = sim_scores_full.merge(ticker_cik_df, how='left', left_on='cik', right_on='cik_short')

# Drop CIK column
#sim_scores_ticker.drop(labels=['cik'], axis='columns', inplace=True)

# Drop NaN values
sim_scores_ticker.dropna(axis='index', how='any', subset=['jaccard_score', 'cosine_score'], inplace=True)

In [495]:
sim_scores_full.head()

,cik,cosine_score,date,jaccard_score
0,1356090,NaN,NaN,NaN
1,1356090,0.868523,NaN,0.766920
2,1356090,0.867265,NaN,0.764305
3,1356090,0.557528,NaN,0.348981
4,1356090,0.587213,NaN,0.348520


In [497]:
sim_scores_ticker

,cik_x,cosine_score,date,jaccard_score,ticker,cik_y,cik_short
1,1356090,0.868523,NaN,0.766920,xon,0001356090,1356090
2,1356090,0.867265,NaN,0.764305,xon,0001356090,1356090
3,1356090,0.557528,NaN,0.348981,xon,0001356090,1356090
4,1356090,0.587213,NaN,0.348520,xon,0001356090,1356090
5,1356090,0.865759,NaN,0.763239,xon,0001356090,1356090
...,...,...,...,...,...,...,...
102803,100885,0.890377,2017-07-20,0.801700,unp,0000100885,100885
102804,100885,0.793234,2017-10-26,0.651991,unp,0000100885,100885
102805,100885,0.867204,2018-04-26,0.765320,unp,0000100885,100885
102806,100885,0.869045,2018-07-19,0.768045,unp,0000100885,100885


In [ ]:
sim_scores_ticker.to_csv('sim_scores_ticker.csv')

## Splitting Dataset

Only run the following cell if reruning notebook:

In [9]:
sim_scores_ticker = pd.read_csv('sim_scores_ticker.csv', index_col = 0)

In [10]:
sim_scores_ticker.drop(labels=['cik_x'], axis='columns', inplace=True)

In [11]:
sim_scores_ticker.rename(columns={'cik_y':'cik'}, inplace=True)

In [14]:
sim_scores_ticker

,cosine_score,date,jaccard_score,ticker,cik,cik_short
1,0.868523,NaN,0.766920,xon,1356090,1356090
2,0.867265,NaN,0.764305,xon,1356090,1356090
3,0.557528,NaN,0.348981,xon,1356090,1356090
4,0.587213,NaN,0.348520,xon,1356090,1356090
5,0.865759,NaN,0.763239,xon,1356090,1356090
...,...,...,...,...,...,...
102803,0.890377,2017-07-20,0.801700,unp,100885,100885
102804,0.793234,2017-10-26,0.651991,unp,100885,100885
102805,0.867204,2018-04-26,0.765320,unp,100885,100885
102806,0.869045,2018-07-19,0.768045,unp,100885,100885


In [71]:
sim_scores_ticker[sim_scores_ticker['ticker'] == 'amzn']

,cosine_score,date,jaccard_score,ticker,cik,cik_short
22857,0.766835,NaN,0.604900,amzn,1018724,1018724
22858,0.765241,NaN,0.606531,amzn,1018724,1018724
22859,0.932932,NaN,0.874264,amzn,1018724,1018724
22860,0.930065,NaN,0.868837,amzn,1018724,1018724
22861,0.946140,NaN,0.897681,amzn,1018724,1018724
22862,0.925157,NaN,0.860683,amzn,1018724,1018724
22866,0.902805,2012-04-27,0.822806,amzn,1018724,1018724
22867,0.906998,2012-07-27,0.829739,amzn,1018724,1018724
22868,0.872645,2012-10-26,0.774005,amzn,1018724,1018724
22869,0.837489,2013-04-26,0.712260,amzn,1018724,1018724


In [15]:
def InitializeEmptyDataframe(start_date, end_date, tickers):
    
    '''
    Initializes an empty DataFrame with all correct indices 
    (1 entry/ticker/day)
    
    Parameters
    ----------
    start_date : datetime.datetime
        Start date of dataframe.
    end_date : datetime.datetime
        End date of dataframe.
    tickers : list
        List of tickers.
    '''
    
    window_length_days = int((end_date - start_date).days)
    date_list = [start_date+timedelta(days=x) for x in range(0, window_length_days)]
    long_date_list = date_list * len(tickers)
    long_date_list = [x.strftime('%Y-%m-%d') for x in long_date_list]
    list.sort(long_date_list)
    empty = pd.DataFrame(data={'date': long_date_list, 
                                     'ticker': tickers*len(date_list),
                                'jaccard_score': [np.nan]*len(tickers)*len(date_list),
                              'cosine_score': [np.nan]*len(tickers)*len(date_list)})
    empty = empty.groupby(['date', 'ticker']).sum()
    
    empty['jaccard_score'] = np.nan
    empty['cosine_score'] = np.nan
    
    return empty

### Create Dataset from 2011-2015

In [75]:
# Initialize empty dataframe
start_date = datetime(2011, 1, 1)
end_date = datetime(2015, 1, 1)
tickers = list(set(sim_scores_ticker['ticker']))

empty_data = InitializeEmptyDataframe(start_date, end_date, tickers)

In [76]:
empty_data.head()

jaccard_score  cosine_score
date       ticker                             
2011-01-01 a                 NaN           NaN
           aa                NaN           NaN
           aal               NaN           NaN
           aamc              NaN           NaN
           aame              NaN           NaN

In [77]:
sim_scores_formatted = sim_scores_ticker.dropna(axis='index', how='any', subset=['jaccard_score', 'cosine_score'])
sim_scores_formatted = sim_scores_formatted.groupby(['date', 'ticker']).agg('mean')

In [78]:
sim_scores_formatted.head()

cosine_score  jaccard_score     cik  cik_short
date       ticker                                                
2011-12-22 camp        0.755000       0.603712  730255     730255
2011-12-30 cag         0.817486       0.691281   23217      23217
           neog        0.662253       0.465062  711377     711377
2012-01-03 bby         0.825220       0.699897  764478     764478
           dri         0.874617       0.776287  940944     940944

In [79]:
sim_scores_formatted.head()

cosine_score  jaccard_score     cik  cik_short
date       ticker                                                
2011-12-22 camp        0.755000       0.603712  730255     730255
2011-12-30 cag         0.817486       0.691281   23217      23217
           neog        0.662253       0.465062  711377     711377
2012-01-03 bby         0.825220       0.699897  764478     764478
           dri         0.874617       0.776287  940944     940944

In [80]:
formatted_data = empty_data.join(sim_scores_formatted, how='left', on=['date', 'ticker'], lsuffix='_empty')
formatted_data.drop(labels=['cosine_score_empty', 'jaccard_score_empty'], axis='columns', inplace=True)

In [81]:
formatted_data.head()

cosine_score  jaccard_score  cik  cik_short
date       ticker                                             
2011-01-01 a                NaN            NaN  NaN        NaN
           aa               NaN            NaN  NaN        NaN
           aal              NaN            NaN  NaN        NaN
           aamc             NaN            NaN  NaN        NaN
           aame             NaN            NaN  NaN        NaN

In [82]:
forward_filled_data = formatted_data.reset_index().sort_values(by=['ticker', 'date'])

In [83]:
forward_filled_data

,date,ticker,cosine_score,jaccard_score,cik,cik_short
0,2011-01-01,a,NaN,NaN,NaN,NaN
3478,2011-01-02,a,NaN,NaN,NaN,NaN
6956,2011-01-03,a,NaN,NaN,NaN,NaN
10434,2011-01-04,a,NaN,NaN,NaN,NaN
13912,2011-01-05,a,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
5067445,2014-12-27,zyne,NaN,NaN,NaN,NaN
5070923,2014-12-28,zyne,NaN,NaN,NaN,NaN
5074401,2014-12-29,zyne,NaN,NaN,NaN,NaN
5077879,2014-12-30,zyne,NaN,NaN,NaN,NaN


In [84]:
forward_filled_data.fillna(method='ffill', limit=90, inplace=True)

In [85]:
forward_filled_data.head()

,date,ticker,cosine_score,jaccard_score,cik,cik_short
0,2011-01-01,a,NaN,NaN,NaN,NaN
3478,2011-01-02,a,NaN,NaN,NaN,NaN
6956,2011-01-03,a,NaN,NaN,NaN,NaN
10434,2011-01-04,a,NaN,NaN,NaN,NaN
13912,2011-01-05,a,NaN,NaN,NaN,NaN


In [86]:
forward_filled_data[forward_filled_data['ticker'] == 'tsla']

,date,ticker,cosine_score,jaccard_score,cik,cik_short
3130,2011-01-01,tsla,NaN,NaN,NaN,NaN
6608,2011-01-02,tsla,NaN,NaN,NaN,NaN
10086,2011-01-03,tsla,NaN,NaN,NaN,NaN
13564,2011-01-04,tsla,NaN,NaN,NaN,NaN
17042,2011-01-05,tsla,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
5067098,2014-12-27,tsla,0.803895,0.665227,1318605.0,1318605.0
5070576,2014-12-28,tsla,0.803895,0.665227,1318605.0,1318605.0
5074054,2014-12-29,tsla,0.803895,0.665227,1318605.0,1318605.0
5077532,2014-12-30,tsla,0.803895,0.665227,1318605.0,1318605.0


In [87]:
del forward_filled_data['cik_short']

In [89]:
forward_filled_data[(forward_filled_data['ticker'] == 'amzn') & (forward_filled_data['date'] >  '2013-01-01')]

,date,ticker,cosine_score,jaccard_score,cik
2546086,2013-01-02,amzn,0.872645,0.774005,1018724.0
2549564,2013-01-03,amzn,0.872645,0.774005,1018724.0
2553042,2013-01-04,amzn,0.872645,0.774005,1018724.0
2556520,2013-01-05,amzn,0.872645,0.774005,1018724.0
2559998,2013-01-06,amzn,0.872645,0.774005,1018724.0
...,...,...,...,...,...
5064158,2014-12-27,amzn,0.887086,0.796856,1018724.0
5067636,2014-12-28,amzn,0.887086,0.796856,1018724.0
5071114,2014-12-29,amzn,0.887086,0.796856,1018724.0
5074592,2014-12-30,amzn,0.887086,0.796856,1018724.0


In [46]:
forward_filled_data.dtypes

date              object
ticker            object
cosine_score     float64
jaccard_score    float64
cik              float64
cik_short        float64
dtype: object

In [50]:
forward_filled_data.to_csv('lazy_prices_data_2011_2015.csv', index=False)

In [520]:
forward_filled_data.to_csv('lazy_prices_data.csv', index=False)

In [52]:
forward_filled_data[forward_filled_data['ticker'] == 'amzn']

,date,ticker,cosine_score,jaccard_score,cik
190,2011-01-01,amzn,NaN,NaN,NaN
3668,2011-01-02,amzn,NaN,NaN,NaN
7146,2011-01-03,amzn,NaN,NaN,NaN
10624,2011-01-04,amzn,NaN,NaN,NaN
14102,2011-01-05,amzn,NaN,NaN,NaN
...,...,...,...,...,...
5064158,2014-12-27,amzn,0.887086,0.796856,1018724.0
5067636,2014-12-28,amzn,0.887086,0.796856,1018724.0
5071114,2014-12-29,amzn,0.887086,0.796856,1018724.0
5074592,2014-12-30,amzn,0.887086,0.796856,1018724.0


### Create Dataset from 2015-2018

In [90]:
# Initialize empty dataframe
start_date = datetime(2015, 1, 1)
end_date = datetime(2018, 1, 1)
tickers = list(set(sim_scores_ticker['ticker']))

empty_data = InitializeEmptyDataframe(start_date, end_date, tickers)

In [91]:
empty_data.head()

jaccard_score  cosine_score
date       ticker                             
2015-01-01 a                 NaN           NaN
           aa                NaN           NaN
           aal               NaN           NaN
           aamc              NaN           NaN
           aame              NaN           NaN

In [92]:
sim_scores_formatted = sim_scores_ticker.dropna(axis='index', how='any', subset=['jaccard_score', 'cosine_score'])
sim_scores_formatted = sim_scores_formatted.groupby(['date', 'ticker']).agg('mean')

In [93]:
sim_scores_formatted.head()

cosine_score  jaccard_score     cik  cik_short
date       ticker                                                
2011-12-22 camp        0.755000       0.603712  730255     730255
2011-12-30 cag         0.817486       0.691281   23217      23217
           neog        0.662253       0.465062  711377     711377
2012-01-03 bby         0.825220       0.699897  764478     764478
           dri         0.874617       0.776287  940944     940944

In [94]:
sim_scores_formatted.head()

cosine_score  jaccard_score     cik  cik_short
date       ticker                                                
2011-12-22 camp        0.755000       0.603712  730255     730255
2011-12-30 cag         0.817486       0.691281   23217      23217
           neog        0.662253       0.465062  711377     711377
2012-01-03 bby         0.825220       0.699897  764478     764478
           dri         0.874617       0.776287  940944     940944

In [95]:
formatted_data = empty_data.join(sim_scores_formatted, how='left', on=['date', 'ticker'], lsuffix='_empty')
formatted_data.drop(labels=['cosine_score_empty', 'jaccard_score_empty'], axis='columns', inplace=True)

In [96]:
formatted_data.head()

cosine_score  jaccard_score  cik  cik_short
date       ticker                                             
2015-01-01 a                NaN            NaN  NaN        NaN
           aa               NaN            NaN  NaN        NaN
           aal              NaN            NaN  NaN        NaN
           aamc             NaN            NaN  NaN        NaN
           aame             NaN            NaN  NaN        NaN

In [97]:
forward_filled_data = formatted_data.reset_index().sort_values(by=['ticker', 'date'])

In [98]:
forward_filled_data

,date,ticker,cosine_score,jaccard_score,cik,cik_short
0,2015-01-01,a,NaN,NaN,NaN,NaN
3478,2015-01-02,a,NaN,NaN,NaN,NaN
6956,2015-01-03,a,NaN,NaN,NaN,NaN
10434,2015-01-04,a,NaN,NaN,NaN,NaN
13912,2015-01-05,a,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
3797975,2017-12-27,zyne,NaN,NaN,NaN,NaN
3801453,2017-12-28,zyne,NaN,NaN,NaN,NaN
3804931,2017-12-29,zyne,NaN,NaN,NaN,NaN
3808409,2017-12-30,zyne,NaN,NaN,NaN,NaN


In [99]:
forward_filled_data.fillna(method='ffill', limit=90, inplace=True)

In [100]:
forward_filled_data.head()

,date,ticker,cosine_score,jaccard_score,cik,cik_short
0,2015-01-01,a,NaN,NaN,NaN,NaN
3478,2015-01-02,a,NaN,NaN,NaN,NaN
6956,2015-01-03,a,NaN,NaN,NaN,NaN
10434,2015-01-04,a,NaN,NaN,NaN,NaN
13912,2015-01-05,a,NaN,NaN,NaN,NaN


In [101]:
forward_filled_data[forward_filled_data['ticker'] == 'tsla']

,date,ticker,cosine_score,jaccard_score,cik,cik_short
3130,2015-01-01,tsla,0.812659,0.684436,1519061.0,1519061.0
6608,2015-01-02,tsla,0.812659,0.684436,1519061.0,1519061.0
10086,2015-01-03,tsla,0.812659,0.684436,1519061.0,1519061.0
13564,2015-01-04,tsla,0.812659,0.684436,1519061.0,1519061.0
17042,2015-01-05,tsla,0.812659,0.684436,1519061.0,1519061.0
...,...,...,...,...,...,...
3797628,2017-12-27,tsla,0.877555,0.781317,1318605.0,1318605.0
3801106,2017-12-28,tsla,0.877555,0.781317,1318605.0,1318605.0
3804584,2017-12-29,tsla,0.877555,0.781317,1318605.0,1318605.0
3808062,2017-12-30,tsla,0.877555,0.781317,1318605.0,1318605.0


In [102]:
del forward_filled_data['cik_short']

In [106]:
forward_filled_data[forward_filled_data['ticker'] == 'tsla']

,date,ticker,cosine_score,jaccard_score,cik
3130,2015-01-01,tsla,0.812659,0.684436,1519061.0
6608,2015-01-02,tsla,0.812659,0.684436,1519061.0
10086,2015-01-03,tsla,0.812659,0.684436,1519061.0
13564,2015-01-04,tsla,0.812659,0.684436,1519061.0
17042,2015-01-05,tsla,0.812659,0.684436,1519061.0
...,...,...,...,...,...
3797628,2017-12-27,tsla,0.877555,0.781317,1318605.0
3801106,2017-12-28,tsla,0.877555,0.781317,1318605.0
3804584,2017-12-29,tsla,0.877555,0.781317,1318605.0
3808062,2017-12-30,tsla,0.877555,0.781317,1318605.0


In [104]:
forward_filled_data.dtypes

date              object
ticker            object
cosine_score     float64
jaccard_score    float64
cik              float64
dtype: object

In [107]:
forward_filled_data.to_csv('lazy_prices_data_2015_2018.csv', index=False)

### Resources
- The dataset of 10-K and 10-Q reports was downloaded from https://sraf.nd.edu/data/stage-one-10-x-parse-data/
- This notebook uses parts from https://www.quantopian.com/posts/scraping-10-ks-and-10-qs-for-alpha